# Clustering com Dados do Cartola FC

## 1.0 Dataset

Foram utilizados dados do Cartola FC obtido através do seguinte [respositório](https://github.com/henriquepgomide/caRtola) do GitHub. O CSV utilizado foi o "dados_agregados_limpos.csv", o qual é a junção dos dados dos anos de 2014 a 2018, além de serem dados já tratados sem a presença de NaN.

Cada linha representa...

Colunas relevantes:
- Rodada: número da rodada do Brasileirão
- ClubeID: clube do jogador
- AtletaID: id do jogador
- Posicao: posição do jogador
- Pontos: pontuação do jogador
- PontosMedia: média da pontuação do jogador
- FS: faltas sofridas
- PE: passes errados
- A: assistências
- FT: finalizações na trave
- FD: finalizações defendidas
- FF: finalizações para fora
- G: gols
- I: impedimentos
- FC: faltas cometidas
- CA: cartões amarelo
- CV: cartões vermelho
- SG: jogos sem sofrer gols
- GS: gols sofridos
- RB: roubadas de bola
- DD: defesas difíceis
- DP: defesas de pênalti

Primeiro, será clusterizado os jogadores separadamente, ou seja, o agrupamento será feito separadamente para cada posição (goleiro, zagueiro, atacante, lateral e meia).

Após essa primeira etapa, será feito...

In [2]:
import pandas as pd

dados_agregados = pd.read_csv('dados_agregados_limpos.csv')
dados_agregados.shape

(17567, 77)

## 2.0 Explorando os dados

### 2.1 Tipos de posição e a quantidade:

In [4]:
dados_agregados.Posicao.value_counts()

mei    6516
ata    4050
lat    3045
zag    2628
gol    1328
Name: Posicao, dtype: int64

### 2.2 Descrição das colunas

In [17]:
colunas = ['Pontos','PontosMedia','FS','PE','A','FT','FF','FC',
           'FD','G','I','RB','CA','CV','SG','DD','DP','GS']

dados_agregados[colunas].describe()

,Pontos,PontosMedia,FS,PE,A,FT,FF,FC,FD,G,I,RB,CA,CV,SG,DD,DP,GS
count,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000,17567.000000
mean,2.946798,2.960628,1.111914,2.315763,0.064325,0.018102,0.347356,1.159845,0.241305,0.090169,0.136847,1.026015,0.169693,0.008140,0.124039,0.115671,0.002505,0.090169
std,4.344379,2.050958,1.304447,2.042131,0.262381,0.135862,0.644317,1.260528,0.538621,0.315378,0.440791,1.265575,0.375980,0.090489,0.329636,0.552542,0.051112,0.435866
min,-10.500000,-8.400000,-3.000000,-6.000000,-1.000000,0.000000,-1.000000,-2.000000,-2.000000,0.000000,-1.000000,-4.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.680000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.880000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.200000,4.090000,2.000000,3.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,33.000000,21.830000,12.000000,14.000000,4.000000,2.000000,7.000000,10.000000,6.000000,3.000000,7.000000,9.000000,1.000000,1.000000,1.000000,7.000000,2.000000,6.000000


### 2.3 Tratar dados duplicados

A problemática é a repetição do valor "AtletaID" nas linhas do dataset, visto que um atleta pode ter participado de várias partidas. Para resolver esse problema, será calculada a média das colunas a serem utilizadas.

- conferir se AtletaID só tem um tipo de posição
- se AtletaID participou de mais de um ClubeID

In [0]:
dados = dados_agregados.copy()
dados = dados[['Rodada','ClubeID','AtletaID','Posicao','Pontos','PontosMedia','FS','PE','A','FT','FF','FC',
           'FD','G','I','RB','CA','CV','SG','DD','DP','GS']]

### 2.4 Separação das posições dos jogadores

Abaixo é feita a filtragem do dataframe de acordo com a posição dos jogaores.

In [15]:
dados_agregados.columns

Index(['A', 'Apelido', 'AtletaID', 'CA', 'CV', 'ClubeID', 'DD', 'DP', 'FC',
       'FD', 'FF', 'FS', 'FT', 'G', 'GC', 'GS', 'I', 'Jogos', 'PE', 'PP',
       'Participou', 'Pontos', 'PontosMedia', 'Posicao', 'Preco',
       'PrecoVariacao', 'RB', 'Rodada', 'SG', 'Status', 'ano', 'avg.A',
       'avg.A.l05', 'avg.CA', 'avg.CV.l05', 'avg.DD', 'avg.DD.l05', 'avg.DP',
       'avg.DP.l05', 'avg.FC', 'avg.FC.l05', 'avg.FD', 'avg.FD.l05', 'avg.FF',
       'avg.FF.l05', 'avg.FS', 'avg.FS.l05', 'avg.FT', 'avg.FT.l05', 'avg.G',
       'avg.G.l05', 'avg.GC', 'avg.GC.l05', 'avg.GS', 'avg.GS.l05', 'avg.I',
       'avg.I.l05', 'avg.PE', 'avg.PE.l05', 'avg.PP', 'avg.PP.l05',
       'avg.Points', 'avg.RB', 'avg.RB.l05', 'avg.SG', 'avg.SG.l05',
       'avg.last05', 'away.score.x', 'dia', 'home.attack', 'home.defend',
       'home.score.x', 'mes', 'pred.away.score', 'pred.home.score',
       'risk_points', 'variable'],
      dtype='object')

In [0]:
dados_meia = dados_agregados[dados_agregados['Posicao']=='mei']

dados_atacante = dados_agregados[dados_agregados['Posicao']=='ata']

dados_lateral = dados_agregados[dados_agregados['Posicao']=='lat']

dados_zaga = dados_agregados[dados_agregados['Posicao']=='zag']

dados_gol = dados_agregados[dados_agregados['Posicao']=='gol']